In [1]:
#Importing Libraries
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#Getting Data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(url)
html = resp.text

In [14]:
#Getting relevant data using BeautifulSoup
soup = BeautifulSoup(html)
pretty_soup = soup.prettify()

### Getting the table from the html using its class('wikitable') and getting td values from every tr(row). Then, converting it into the dataframe.

In [15]:
table = soup.find('table', attrs={'class':'wikitable'})
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
        
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [16]:
#remove rows where Borough is 'not assigned'
df = df[df['Borough']!='Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [17]:
#If neighbourhood is 'not assigned', then the neighborhood will be the same as the borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned',df['Borough'],df['Neighbourhood'])

In [18]:
#If Postal Code is same
df = df.groupby('PostalCode').agg({'Neighbourhood': ', '.join,'Borough':'first'})[['Neighbourhood','Borough']].reset_index()

In [20]:
df.shape

(103, 3)